# TensorBoard with Fashion MNIST

다음 exercise를 따라한 것임
* https://www.coursera.org/learn/advanced-deployment-scenarios-tensorflow/ungradedLab/LvDtn/tensorboard-assignment/lab



In [1]:
# load tensorboard notebook extension
%load_ext tensorboard

In [2]:
import io
import itertools
import numpy as np
import sklearn.metrics
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from datetime import datetime

from os import getcwd


print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.3.0


In [17]:
# Load Fashion-MNIST from tfds
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Names of the integer classes, i.e., 0 -> T-short/top, 1 -> Trouser, etc.
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## Visualizing a single Image in tensorboard

In [18]:
print("Shape: ", train_images[0].shape)
print("Label: ", train_labels[0], "->", class_names[train_labels[0]])

Shape:  (28, 28)
Label:  9 -> Ankle boot


In [5]:
# reshape image for the summary API 
img = np.reshape(train_images[0], (-1, 28, 28, 1))

In [6]:
# clear out log data
!rm -rf logs

logdir = "logs/train_data/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir)

with file_writer.as_default():
    tf.summary.image("Training data", img, step=0)

In [8]:
%tensorboard --logdir logs/train_data --port=6007

## Visualizing multiples image in tensorboard

In [10]:
with file_writer.as_default():
    images = np.reshape(train_images[0:10], (-1, 28, 28, 1))
    tf.summary.image('10 training data examples', images, max_outputs=25, step=0)

%tensorboard --logdir logs/train_data --port=6007

Reusing TensorBoard on port 6007 (pid 48910), started 0:00:29 ago. (Use '!kill 48910' to kill it.)

## Logging arbitaray image data

In [16]:
!rm -rf logs/plots

logdir = "logs/plots/" + datetime.now().strftime('%Y%m%d-%H%M%S')
file_writer = tf.summary.create_file_writer(logdir)

def plot_to_image(figure):
    "Convert matplotlib plot by figure to a PNG image"
    
    # convert to png 
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    
    # close figure so as to prevent it from being displayed
    plt.close(figure)
    
    # convert png to TF image
    buf.seek(0)
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)  # add batch dimension
    
    return image


def image_grid():
    " 5x5 grid of the images as matplotlib figure"
    
    figure = plt.figure(figsize=(10,10))
    for i in range(25):
        plt.subplot(5, 5, i+1, title=class_names[train_labels[i]])
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(train_images[i], cmap=plt.cm.binary)
    
    return figure

# Prepare the plot
figure = image_grid()

with file_writer.as_default():
    tf.summary.image("Training data in grid", plot_to_image(figure), step=0)
    
%tensorboard --logdir logs/plots

## Build Classifier

In [25]:
# Build the model
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Plot confusion matrix during training

In [19]:
def plot_confusion_matrix(cm, class_names):
    
    
    figure = plt.figure(figsize=(8,8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    
    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    return figure

In [20]:
!rm -rf logs/image
logdir = "logs/image/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir+'/cm')

In [27]:
def log_confusion_matrix(epoch, logs):
    
    # predict on validation dataset
    test_pred_raw = model.predict(test_images)
    test_pred = np.argmax(test_pred_raw, axis=1)
    
    # calculate confusion matrix
    cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
    
    # log cm as image summary
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)
    
    with file_writer_cm.as_default():
        tf.summary.image('Confusion matrix', cm_image, step=epoch)

cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)        

In [28]:
# Start TensorBoard.
%tensorboard --logdir logs/image

# Train the classifier.
model.fit(
    train_images,
    train_labels,
    epochs=5,
    verbose=0, # Suppress chatty output
    callbacks=[tensorboard_callback, cm_callback],
    validation_data=(test_images, test_labels),
)

Reusing TensorBoard on port 6008 (pid 61091), started 0:03:04 ago. (Use '!kill 61091' to kill it.)